In [60]:
from PIL import Image
import numpy as np
import networkx as nx

"""
CREATING THE PIXEL GRAPH AND DEFINING SIGNALS AS SIMILARITY TO ADJACENT NODES
"""

image_path = 'assets/pomeranian.jpg'
resized_image_path = 'assets/resized_pomeraniam.jpg'

# loading the image
image = Image.open(image_path).convert('L')
low_res_image = image.resize((50, 50))  # Resize to 128x128 pixels
low_res_image.save(resized_image_path)
print("image type:", image.format)

# converting to array
data = np.asarray(low_res_image, dtype=np.int16)
print("array shape:", data.shape)
print("Data", data)


# creating a graph using pixel intensities
G = nx.Graph()

def absoluteDifference(curr, data, neighbor_id):
    return np.abs(curr - data[neighbor_id[0]][neighbor_id[1]])

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        curr = data[i][j]
        curr_id = (i, j)
        # up
        if i > 0:
            up_id = (i - 1, j)
            w = absoluteDifference(curr, data, up_id)
            G.add_edge(curr_id, up_id, weight=w)
        # down
        if i < data.shape[0] - 1:
            down_id = (i + 1, j)
            w = absoluteDifference(curr, data, down_id)
            G.add_edge(curr_id, down_id, weight=w)
        # left
        if j > 0:
            left_id = (i, j - 1)
            w = absoluteDifference(curr, data, left_id)
            G.add_edge(curr_id, left_id, weight=w)
        # right
        if j > data.shape[1] - 1:
            right_id = (i, j + 1)
            w = absoluteDifference(curr, data, right_id)
            G.add_edge(curr_id, right_id, weight=w)  

image type: None
array shape: (50, 50)
Data [[157 165 160 ... 144 153 157]
 [158 165 161 ... 160 166 162]
 [159 164 164 ... 165 168 161]
 ...
 [142 133 139 ... 159 159 168]
 [137 130 138 ... 157 158 160]
 [141 142 138 ... 162 159 157]]


In [56]:
"""
CONSTRUCTING THE GRAPH LAPLACIAN AND COMPUTING EIGENVALUES/EIGENVECTORS 
"""
import scipy.linalg as splin

# constructing the graph laplacian L = D - A
L = nx.linalg.normalized_laplacian_matrix(G)
L_dense = L.toarray()
ews, evs = splin.eigh(L_dense)
print("Laplacian shape", L.shape)
print("Eigenvalues shape:", ews.shape)
print("Eigenvectors shape:", evs.shape)

Laplacian shape (2500, 2500)
Eigenvalues shape: (2500,)
Eigenvectors shape: (2500, 2500)


In [57]:
"""
TRANSFORMING GRAPH SIGNALS INTO THE SPECTRAL DOMAIN
"""
X = data.flatten()
print("eigenvector matrix:", evs.shape)
print("X:", X.shape)

X_spectral = evs.transpose().dot(X)
print("X (spectral):", X_spectral.shape)

print(X_spectral)

eigenvector matrix: (2500, 2500)
X: (2500,)
X (spectral): (2500,)
[-5.72499595e+03 -3.37146640e+02 -1.41211840e+03 ...  1.59255078e+01
  8.91154023e-01 -2.02069076e+01]


In [58]:
"""
APPLYING A HIGH PASS GRAPH FILTER ONTO THE IMAGE AND INVERTING THE GFT
"""
tau = 0.005
H = np.diag(np.where(ews > tau, 1, 0))
X_filtered_spectral = H @ X_spectral

# Transforming the filtered signal back to the spacial domain
X_filtered_spatial = evs @ X_filtered_spectral
filtered_image = X_filtered_spatial.reshape(data.shape)

print("X (filtered-spatial):", X_filtered_spatial.shape)
print("Filtered Image):", filtered_image.shape)


X (filtered-spatial): (2500,)
Filtered Image): (50, 50)


In [59]:
"""
SAVING THE FILTERED IMAGE
"""
filtered_image_pil = Image.fromarray(filtered_image.astype(np.uint8))  # convert to PIL image
filtered_image_pil.save('assets/filtered_pomeranian.jpg')